
# Trabajo semanal N°13 - Síntesis de redes doblemente cargadas

### Autor: Gonzalo Goumaz



## Consigna

Diseñe el cuadripolo A para que se comporte como:

<ul>
    <li> pasa bajos Bessel de 3er. orden
    <li> item no disipativo
    <li> normalizado en frecuencia e impedancia
</ul>   
respetando la siguiente topología:

<figure>
<center>
<img src="Images/consigna.png">
<figcaption> Fig 1.1: Topología a sintetizar.</figcaption>
</figure>
    

1.  Obtenga la impedancia de entrada al cuadripolo A, cargado con un resistor de 1Ω a la salida.
2.  Sintetice A como un cuadripolo escalera.
3.  Simule el comportamiento de la red en LTspice graficando S21 y S11 en función de la frecuencia. (Ver explicación de Agustín Alba Chicar 1h 48m)
4.  Explique el comportamiento de A a partir de los valores de S11 en las siguientes frecuencias:
    <ul>
        <li> centro de la banda de paso
        <li> frecuencia de corte
        <il> transición y centro de la banda de detenida
    </ul>
5.  Modifique el circuito para que la frecuencia de corte sea 2 π 10⁶ rad/s y la resistencia del generador sea 50Ω.



## Resolución analítica

En el PDF adjunto a continuación se encuentra desarrollada la síntesis.

In [1]:
from IPython.display import IFrame
IFrame("TS13.pdf", width = 900, height = 900)


### Cálculos auxiliares en Python

A continuación se encuentra el código utilizado para realizar la síntesis anterior.

In [4]:
import sympy as sp
from pytc2.general import print_latex, a_equal_b_latex_s
from pytc2.remociones import modsq2mod_s
import pytc2.cuadripolos as tc2

# ---------------------------------------- #

s = sp.symbols('s', complex=True) # Defino la variable compleja S

# ---------------------------------------- #

# ------------- Calculos utilizados para desarrollo -------------- #

s21 = sp.Rational('15')/(s**3+ sp.Rational('6')*s**2 + sp.Rational('15')*s + sp.Rational('15'))
module_s21_square = s21 * s21.subs(s, -s) 

module_s11_square = sp.factor(1 - module_s21_square)
s11 = modsq2mod_s(module_s11_square)

print("El parámetro s11(s) a sintetizar resulta")
print_latex(a_equal_b_latex_s('s11(s)', s11))

El parámetro s11(s) a sintetizar resulta


<IPython.core.display.Math object>


## Impedancia de entrada

Se utiliza Python para obtener la impedancia de entrada.

In [21]:
# ---------Impedancia de entrada---------- #

# Armo la MAI
#                     2 
# 0----Rg----L1---------------L2-----------  3  #
#                     -                 -       #
#                     -                 -       #
#                     C                 RL      #
#                     -                 -       #
#                     -                 -       #
# 1----------------------------------------  1  #

Rg = sp.nsimplify(1)
RL = sp.nsimplify(1)
C  = sp.nsimplify(0.552)
L1 = sp.nsimplify(1.2547)
L2 = sp.nsimplify(0.1925)



Ymai = sp.Matrix([
        [1/(Rg + s*L1), 0, -1/(Rg+ s*L1), 0],
        [0, s*C + 1/RL, -s*C, -1/RL],
        [-1/(Rg + s*L1), -s*C, s*C + 1/(Rg + s*L1) + 1/(s*L2), -1/(s*L2)],
        [0, -1/RL, -1/(s*L2), 1/RL + 1/(s*L2)]
        
    ])
 
Zin = tc2.calc_MAI_impedance_ij(Ymai, ii=0, jj=1)

print("La impedancia de entrada de la red está dada por:")
print_latex(a_equal_b_latex_s("Z_in(s)", sp.factor(Zin)));

# ---------------------------------------- #

La impedancia de entrada de la red está dada por:


<IPython.core.display.Math object>


## Red simulada en LTspice

Se realiza una simulación circuital de los parámetros S11 y S21 en LTspice.

<figure>
<center>
<img src="Images/red_spice.png">
<figcaption> Fig 2.1: Draft de simulación en LTspice.</figcaption>
</figure>
    
<figure>
<center>
<img src="Images/sim_spice.png">
<figcaption> Fig 2.2: Simulación de $S_{11}$ y $S_{21}$.</figcaption>
</figure>
    
Se observa que hasta la pulsación angular unitaria el retardo de grupo es constante. Lo cual indica que el filtro adhiere a una aproximación de tipo Bessel.


## Análisis de $A$ según $S_{11}$

Se tiene que $s_{11}$

\begin{equation}
s_{11}(s)=\frac{s \left(s^{2} + 2 \sqrt{3} \cdot \sqrt[4]{5} s \cos{\left(\frac{\operatorname{atan}{\left(2 \right)}}{2} \right)} + 3 \sqrt{5}\right)}{s^{3} + 6 s^{2} + 15 s + 15}
\end{equation}

Se observa que para $s=0$ se tiene un cero, por lo que no existe reflexión de la onda. Entonces, se puede decir que el cuadripolo permite el paso de frecuencias bajas.
<br>
En el otro extremo, cuando $s\rightarrow\infty$ se tiene que $s_{11}=1$. Entonces, toda la onda se refleja. Se puede afirmar que el cuadripolo no permite el paso de las altas frecuencias.
<br>
Con este análisis se puede afirmar que el compartamiento del filtro es de tipo pasa bajos.


## Desnormalización de la red

Se desea analizar el filtro para una frecuencia de operación de $f_0 = 1Mhz$ y una $R_g=50\Omega$

<figure>
<center>
<img src="Images/red_spice_2.png">
<figcaption> Fig 3.1: Draft de simulación en LTspice.</figcaption>
</figure>
    
<figure>
<center>
<img src="Images/sim_spice_2.png">
<figcaption> Fig 3.2: Simulación de $S_{11}$ y $S_{21}$.</figcaption>
</figure>
    

Se observa que el ancho de banda para el cual se mantiene constante el retardo de grupo coincide con la frecuencia de operación